In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
import os

# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256"
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64"

_LOCAL_DATA_DR = "data"
_LOCAL_MODL_DR = "models"
_DATASET_FNAME = "birdsnapLite_Final"
_IMG_SIZ       = (64, 64)

try:
    from google.colab import drive
    _DATASET_GPATH = os.path.join( "/content/drive/MyDrive", _DATASET_FNAME )
    _WEIGHTS_GPATH = os.path.join( "/content/drive/MyDrive", _LOCAL_MODL_DR )
    drive.mount( '/content/drive' )
    print( "Google Drive MOUNTED!" )
    %%capture
    !pip install datasets
    !pip install evaluate
    !pip install accelerate -U
    !pip install peft
except Exception as e:
    print( f"{e}: We are running locally" )
    _DATASET_GPATH = os.path.join( _LOCAL_DATA_DR, _DATASET_FNAME )
    _WEIGHTS_GPATH = os.path.join( _LOCAL_DATA_DR, _LOCAL_MODL_DR )
_WEIGHTS_GPATH = os.path.join( _WEIGHTS_GPATH, "clip.pt" )

p_DataSetOnDrive = os.path.isdir( _DATASET_GPATH )

if p_DataSetOnDrive:
  print( f"{_DATASET_FNAME} was found on your disk!" )
else:
  print( f"Birdsnap Lite Dataset (\"{_DATASET_FNAME}\") will be downloaded from Hugging Face and saved to your Google Drive!" )

No module named 'google.colab': We are running locally
Birdsnap Lite Dataset ("birdsnapLite_Final") will be downloaded from Hugging Face and saved to your Google Drive!


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms as T
from datasets import load_dataset
from transformers import AutoImageProcessor, ViTForImageClassification
from transformers import Trainer, TrainingArguments
import evaluate

In [4]:
from datasets import Dataset, load_dataset, load_from_disk
import json
import torchvision.transforms as T


dataset = None
if not p_DataSetOnDrive:
    dataset = load_dataset( "jwatson-CO-edu/birdsnap_lite" )
else:
    dataset = load_from_disk( _DATASET_GPATH )

# dataset['train'][0]

In [5]:
classes = dataset['train'].unique( 'common' )
print(len(classes))

500


In [6]:
from transformers import Owlv2Processor, Owlv2ForObjectDetection

# model_name       = "owlv2-base-patch16-ensemble"
model_name       = "owlv2-base-patch16"
model_checkpoint = f"google/{model_name}"

processor = Owlv2Processor.from_pretrained( 
    model_checkpoint,
    do_rescale=False
)
processor

preprocessor_config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

Owlv2Processor:
- image_processor: Owlv2ImageProcessor {
  "do_normalize": true,
  "do_pad": true,
  "do_rescale": false,
  "do_resize": true,
  "image_mean": [
    0.48145466,
    0.4578275,
    0.40821073
  ],
  "image_processor_type": "Owlv2ImageProcessor",
  "image_std": [
    0.26862954,
    0.26130258,
    0.27577711
  ],
  "processor_class": "Owlv2Processor",
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 960,
    "width": 960
  }
}

- tokenizer: CLIPTokenizerFast(name_or_path='google/owlv2-base-patch16', vocab_size=49408, model_max_length=16, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|startoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '!'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("!", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	49406: AddedToken("<|startoftext|>", rstrip=False, lstrip

In [7]:
label2id = {c:idx for idx,c in enumerate(classes)}
id2label = {idx:c for idx,c in enumerate(classes)}

In [8]:
dataset = dataset['train'].train_test_split(train_size=0.8)
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'common', 'scientific', 'common_idx', 'scientific_idx'],
        num_rows: 30476
    })
    test: Dataset({
        features: ['image', 'common', 'scientific', 'common_idx', 'scientific_idx'],
        num_rows: 7620
    })
})

In [9]:
def enforce_RGB( img ):
    img = torch.tensor( img )
    # Stack greyscale images
    if img.shape[0] == 1:
        swp = torch.zeros( 3, _IMG_SIZ[0], _IMG_SIZ[0] )
        for j in range(3):
            swp[j] = img[0,:,:]
        img = swp
    # I guess some pix have an ALPHA CHANNEL????
    if img.shape[0] > 3:
        img = img[:3,:,:]
    return img.clone()

def transforms( batch ):
    batch['image'] = [enforce_RGB( x ) for x in batch['image']]
    # batch['common'] = [x.split() for x in batch['common']]
    # batch['common'] = [str( x ).replace('\'','') for x in batch['common']]
    # print( batch['label'] )
    # print( batch['common'] )
    # del batch['common_idx']
    # del batch['scientific_idx']
    # batch['common'] = [str(elem).replace('\'','').split() for elem in batch['common']]
    # batch['scientific'] = [str(elem).replace('\'','').split() for elem in batch['scientific']]
    # print( batch.keys() )
    # print( batch['scientific'] )
    # inputs = processor([x for x in batch['image']],return_tensors='pt')
    inputs = processor(text=batch['common'], images=[x for x in batch['image']], return_tensors="pt")
    inputs['labels'] = [label2id[y] for y in batch['common']]
    return inputs

In [10]:
dataset = dataset.with_transform( transforms )

In [11]:
# def collate_fn( batch ):
#     return {
#         'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
#         'common': torch.tensor([x['common'] for x in batch])
#         # 'common': torch.tensor([[x['common'],] for x in batch])
#     }

def collate_fn( batch ):
    # print( batch[0].keys() )
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        # 'labels': torch.tensor([x['labels'] for x in batch])
        # 'labels': torch.tensor([x['common'] for x in batch])
        # 'common': torch.tensor([x['labels'] for x in batch])
        # 'label': torch.tensor([x['labels'] for x in batch])
        'input_ids': torch.tensor([x['labels'] for x in batch])
    }

In [12]:
accuracy = evaluate.load( 'accuracy' )
def compute_metrics( eval_preds ):
    logits, labels = eval_preds
    predictions = np.argmax(logits,axis=1)
    acc = accuracy.compute(predictions=predictions, references=labels)
    return acc

In [13]:
def print_trainable_parameters( model ):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [14]:
model = Owlv2ForObjectDetection.from_pretrained( model_checkpoint )

print_trainable_parameters( model )

config.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/620M [00:00<?, ?B/s]

trainable params: 154966792 || all params: 154966792 || trainable%: 100.00


In [15]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    target_modules=["q_proj","v_proj"], 
    inference_mode=False, 
    r=8, 
    lora_alpha=32, 
    lora_dropout=0.05,
    bias="none",
    modules_to_save=["classifier"],
    # do_rescale=False    
)

lora_model = get_peft_model( model, config )
print_trainable_parameters( lora_model )

trainable params: 491520 || all params: 155458312 || trainable%: 0.32


In [16]:
model

Owlv2ForObjectDetection(
  (owlv2): Owlv2Model(
    (text_model): Owlv2TextTransformer(
      (embeddings): Owlv2TextEmbeddings(
        (token_embedding): Embedding(49408, 512)
        (position_embedding): Embedding(16, 512)
      )
      (encoder): Owlv2Encoder(
        (layers): ModuleList(
          (0-11): 12 x Owlv2EncoderLayer(
            (self_attn): Owlv2Attention(
              (k_proj): Linear(in_features=512, out_features=512, bias=True)
              (v_proj): lora.Linear(
                (base_layer): Linear(in_features=512, out_features=512, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=512, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=512, bias=False)
                )
                (lora_embedd

In [17]:
batch_size = 16 #32 # 64 # 128

args = TrainingArguments(
    f"{model_checkpoint}-finetuned-lora-birdsnap-lite",
    per_device_train_batch_size=batch_size,
    learning_rate=5e-3,
    num_train_epochs=5,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=4,
    logging_steps=10,
    save_total_limit=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="accuracy",
    report_to='tensorboard',
    fp16=True,
    push_to_hub=True,
    remove_unused_columns=False,
    load_best_model_at_end=True,
    # do_rescale=False
)

/home/james/.local/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
trainer = Trainer(
    model=model,
    args=args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor,
    # do_rescale=False
)

In [20]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 9.28 GiB. GPU 0 has a total capacity of 7.78 GiB of which 1.25 GiB is free. Including non-PyTorch memory, this process has 6.30 GiB memory in use. Of the allocated memory 6.15 GiB is allocated by PyTorch, and 38.76 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
repo_name = f"jwatson-CO-edu/{model_name}-finetuned-lora-birdsnap-lite"
lora_model.push_to_hub( repo_name )

In [ ]:
from peft import PeftConfig, PeftModel


config = PeftConfig.from_pretrained(repo_name)
model = AutoModelForImageClassification.from_pretrained(
    config.base_model_name_or_path,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)
# Load the Lora model
inference_model = PeftModel.from_pretrained( model, repo_name )

In [ ]:
import requests
from PIL import Image
import torch

from transformers import Owlv2Processor, Owlv2ForObjectDetection

processor = Owlv2Processor.from_pretrained("google/owlv2-base-patch16-ensemble")
model = Owlv2ForObjectDetection.from_pretrained("google/owlv2-base-patch16-ensemble")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
texts = [["a photo of a cat", "a photo of a dog"]]
inputs = processor(text=texts, images=image, return_tensors="pt")
outputs = model(**inputs)

# Target image sizes (height, width) to rescale box predictions [batch_size, 2]
target_sizes = torch.Tensor([image.size[::-1]])
# Convert outputs (bounding boxes and class logits) to Pascal VOC Format (xmin, ymin, xmax, ymax)
results = processor.post_process_object_detection(outputs=outputs, target_sizes=target_sizes, threshold=0.1)
i = 0  # Retrieve predictions for the first image for the corresponding text queries
text = texts[i]
boxes, scores, labels = results[i]["boxes"], results[i]["scores"], results[i]["labels"]
for box, score, label in zip(boxes, scores, labels):
    box = [round(i, 2) for i in box.tolist()]
    print(f"Detected {text[label]} with confidence {round(score.item(), 3)} at location {box}")